In [ ]:
import pandas as pd

In [ ]:
c = [x for x in range(224*224)]

In [ ]:
import tensorflow as tf

In [ ]:
# Loading the packages 

from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import ZeroPadding2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization,Input
from tensorflow.keras.models import Model
from tensorflow.keras import layers
import tensorflow as tf



# Creating the same training resnet model
def identity_block(input_tensor, kernel_size, filters):
    
    filters1, filters2, filters3 = filters
    
    

    x = Conv2D(filters1,1)(input_tensor)
    x = BatchNormalization()(x)
    x = Activation(tf.keras.layers.LeakyReLU(alpha=0.3))(x)

    x = Conv2D(filters2, kernel_size,padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(tf.keras.layers.LeakyReLU(alpha=0.3))(x)

    x = Conv2D(filters3, 1)(x)
    x = BatchNormalization()(x)

    x = layers.add([x, input_tensor])
    x = Activation(tf.keras.layers.LeakyReLU(alpha=0.3))(x)
    return x

def conv_block(input_tensor, kernel_size, filters, strides=(2, 2)):
    
    
    filters1, filters2, filters3 = filters

    x = Conv2D(filters1, 1, strides=strides)(input_tensor)
    x = BatchNormalization()(x)
    x = Activation(tf.keras.layers.LeakyReLU(alpha=0.3))(x)

    x = Conv2D(filters2, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(tf.keras.layers.LeakyReLU(alpha=0.3))(x)

    x = Conv2D(filters3,1)(x)
    x = BatchNormalization()(x)

    shortcut = Conv2D(filters3,  1, strides=strides)(input_tensor)
    shortcut = BatchNormalization()(shortcut)

    x = layers.add([x, shortcut])
    x = Activation(tf.keras.layers.LeakyReLU(alpha=0.3))(x)
    return x
inputs = Input(shape=(224,224,1))

x = ZeroPadding2D((3, 3))(inputs)
x = Conv2D(64, (7, 7), strides=(2, 2))(x)
x = BatchNormalization()(x)
x = Activation(tf.keras.layers.LeakyReLU(alpha=0.3))(x)
x = MaxPooling2D((3, 3), strides=(2, 2))(x)

x = conv_block(x, 3, [64, 64, 256],  strides=(1, 1))
x = identity_block(x, 3, [64, 64, 256] )
x = identity_block(x, 3, [64, 64, 256])

x = conv_block(x, 3, [128, 128, 512])
x = identity_block(x, 3, [128, 128, 512])
x = identity_block(x, 3, [128, 128, 512])
x = identity_block(x, 3, [128, 128, 512])

x = conv_block(x, 3, [256, 256, 1024])
x = identity_block(x, 3, [256, 256, 1024])
x = identity_block(x, 3, [256, 256, 1024])
x = identity_block(x, 3, [256, 256, 1024])
x = identity_block(x, 3, [256, 256, 1024])
x = identity_block(x, 3, [256, 256, 1024])

x = conv_block(x, 3, [512, 512, 2048])
x = identity_block(x, 3, [512, 512, 2048])
x = identity_block(x, 3, [512, 512, 2048])

x = AveragePooling2D((7, 7))(x)

   
x = Flatten()(x)
x = Dense(28, activation='softmax')(x)

resmodel = Model(inputs, x)
resmodel.compile(loss='categorical_crossentropy', optimizer='Adamax', metrics=['accuracy'])

In [ ]:
shape = 224

In [ ]:
# Loading the trained weights
path = "../input/klatask2/task2resnet50(m)leakyrelu (1).h5"

resmodel.load_weights(path)

In [ ]:

# Decoded labels from the model
d = {'0.0': 0,
 '1.0': 1,
 '11.0': 2,
 '12.0': 3,
 '13.0': 4,
 '14.0': 5,
 '15.0': 6,
 '16.0': 7,
 '17.0': 8,
 '18.0': 9,
 '19.0': 10,
 '2.0': 11,
 '20.0': 12,
 '21.0': 13,
 '22.0': 14,
 '23.0': 15,
 '24.0': 16,
 '25.0': 17,
 '26.0': 18,
 '27.0': 19,
 '28.0': 20,
 '3.0': 21,
 '4.0': 22,
 '5.0': 23,
 '6.0': 24,
 '7.0': 25,
 '8.0': 26,
 '9.0': 27}

In [ ]:
ind = {v: k for k, v in d.items()}

In [ ]:

# Loading the test data and converting into numpy arrays and passing to the trained models and making predictions 
import matplotlib.pyplot as plt
from skimage.transform import rescale, resize
from tqdm import tqdm 
import numpy as np
out = []
for x in tqdm(range(0,1365)) :
    try :
        df = pd.read_csv("../input/klatask2/test_img_2d.csv",sep = ',',skiprows = x ,nrows = 1,names = c) # loading csv file
 
        df.columns = c
        
        a = df.iloc[0,c].values
#         print(a)
        b = a.reshape(shape,shape)
#         print(b)
        b  = b.reshape((1,shape,shape,1))
        
        b = b /255 
        o = resmodel.predict(b)
        out.append(float(ind[np.argmax(o)]))
        
        print(float(ind[np.argmax(o)]))
       
    except :
        out.append(1.0)
        
    
    
    
    
    

In [ ]:
df = pd.read_csv("../input/klatask2/test_img_2d.csv",sep = ',',skiprows = 0 ,nrows = 1,names = c)

df.columns = c
        
a = df.iloc[0,c].values
print(a)

In [ ]:
len(a)

In [ ]:
a = a.reshape((shape,shape))

In [ ]:
a.shape

In [ ]:
a = a.reshape((1,shape,shape,1))

In [ ]:
df0 = pd.DataFrame()   #Converting to csv
df0["Output"] = out
df0.to_csv("Task-2 Lables.csv",index  = None)

In [ ]:
df0